In [1]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """

    recs = np.random.choice(items_weights['item_id'].unique(), p=items_weights['weight'], size=n, replace=False)
    
    return recs.tolist()

In [6]:
def create_items_weights(data):
    items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
    items_weights += 1
    log_sales_value = items_weights['sales_value'].apply(lambda x: np.log(x))
    log_sales_value_sum = log_sales_value.sum()
    items_weights['weight'] = log_sales_value.apply(lambda x: x / log_sales_value_sum)
    items_weights = items_weights.drop(columns='sales_value')
    
    return items_weights

In [7]:
items_weights = create_items_weights(data_train)
items_weights.head(2)

,item_id,weight
0,25672,0.000013
1,26082,0.000003


In [8]:
items_weights['weight'].sum()

1.0

In [9]:
%%time

weighted_random_recs = weighted_random_recommendation(items_weights, n=5)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recs)
result.head(2)

Wall time: 5 ms


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1182677, 1133019, 8116838, 6442856, 69073]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1182677, 1133019, 8116838, 6442856, 69073]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

#### Считаем на топ-5000 товаров

In [10]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)['item_id'].tolist()

In [11]:
data_train.loc[(~data_train['item_id'].isin(top_5000)), 'item_id'] = 1111111

In [12]:
data_train['item_id'].nunique()

5001

In [13]:
data_train_5000 = data_train.loc[(data_train['item_id'] != 1111111)]

In [14]:
items = data_train_5000['item_id'].unique()

In [15]:
len(items)

5000

In [16]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()


def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

##### random_recommendation_5000

In [17]:
%%time

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 208 ms


,user_id,actual,weighted_random_recommendation,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1182677, 1133019, 8116838, 6442856, 69073]","[1053690, 883418, 901543, 1092948, 1060705]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1182677, 1133019, 8116838, 6442856, 69073]","[843171, 862139, 941734, 1021152, 13511722]"


##### popularity_recommendation_5000

In [18]:
%%time

popular_recs = popularity_recommendation(data_train_5000, n=5)

result['popular_recommendation_5000'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 51 ms


,user_id,actual,weighted_random_recommendation,random_recommendation_5000,popular_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1182677, 1133019, 8116838, 6442856, 69073]","[1053690, 883418, 901543, 1092948, 1060705]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1182677, 1133019, 8116838, 6442856, 69073]","[843171, 862139, 941734, 1021152, 13511722]","[6534178, 6533889, 1029743, 6534166, 1082185]"


##### weighted_random_recommendation_5000

In [19]:
items_weights = create_items_weights(data_train_5000)
items_weights.head(2)

,item_id,weight
0,202292,0.000147
1,397897,0.000265


In [20]:
items_weights['weight'].sum()

0.9999999999999999

In [21]:
%%time

weighted_random_recs = weighted_random_recommendation(items_weights, n=5)

result['weighted_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recs)
result.head(2)

Wall time: 3 ms


,user_id,actual,weighted_random_recommendation,random_recommendation_5000,popular_recommendation_5000,weighted_random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1182677, 1133019, 8116838, 6442856, 69073]","[1053690, 883418, 901543, 1092948, 1060705]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8090561, 958068, 894426, 1103477, 834104]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1182677, 1133019, 8116838, 6442856, 69073]","[843171, 862139, 941734, 1021152, 13511722]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8090561, 958068, 894426, 1103477, 834104]"


#### Выбираем число соседей $K$ для ItemItemRecommender

In [22]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведём в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [23]:
# создаём словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [24]:
%%time

for k in range(1, 11):
    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей

    model.fit(sparse_user_item.T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    result['itemitem_k' + str(k)] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]] 
                                                                             for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                                                        user_items=sparse_user_item, 
                                                                                                        N=5, 
                                                                                                        filter_already_liked_items=False, 
                                                                                                        filter_items=None, 
                                                                                                        recalculate_user=True)
                                                                            ])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

...

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 3.31 s


In [25]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(recommended_list)

In [26]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(), 8)}:{name_col}")

1.0:actual
9.794e-05:weighted_random_recommendation
0.00538688:random_recommendation_5000
0.15523996:popular_recommendation_5000
0.00039177:weighted_random_recommendation_5000
0.14277669:itemitem_k1
0.19200947:itemitem_k2
0.1870715:itemitem_k3
0.14495593:itemitem_k4
0.13692458:itemitem_k5
0.14201763:itemitem_k6
0.14485798:itemitem_k7
0.14720862:itemitem_k8
0.14848188:itemitem_k9
0.15093046:itemitem_k10


#### Выводы

weighted_random_recommendation показал худшее качество из всех метрик, особенно на топ-5000.

popular_recommendation на топ-5000 такой же, как без топ-5000.

random_recommendation на топ-5000 лучше, чем без.

Лучшее качество у ItemItemRecommender наблюдается при K=2.